In [1]:
import json

import pandas as pd
import pickle

In [2]:
from utils.eval_utils import *

In [3]:
def get_pckl_keys(dir_path):
    """

    Function to show the keys in the h5py file.

    """

    for scale in os.listdir(dir_path):
        for location in os.listdir(os.path.join(dir_path, scale)):
            print(location)
            if location.endswith(".pkl"):
                with open(os.path.join(dir_path, scale, location), 'rb') as f:
                    data = pickle.load(f)
                    print(f"Horizons: {data.keys()}")
                    print(f"Seasons: {data[list(data.keys())[0]].keys()}")
                    print(f"Models: {data[1]['Winter'][0].keys()}")

                    return data
                    

In [4]:
dict_result_n_ahead = get_pckl_keys("/Users/nikolaushouben/Desktop/WattCast/data/evaluations")

df_metrics_building_1.csv
building_1.pkl
Horizons: dict_keys([1, 4, 8, 24, 48])
Seasons: dict_keys(['Winter', 'Summer'])
Models: dict_keys(['RandomForest', 'XGBModel', 'LightGBMModel', 'BlockRNNModel', 'NBEATSModel', 'TFTModel', 'LinearRegressionModel'])


In [5]:
from utils.pipeline import Config, derive_config_params

with open("init_config.json", "r") as f:
    init_config = json.load(f)


config = Config().from_dict(init_config)

config = derive_config_params(config)



In [76]:
# metrics 
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
rmse = lambda df: mean_squared_error(df.iloc[:, 1], df.iloc[:, 0], squared=False)
mae = lambda df: mean_absolute_error(df.iloc[:, 1], df.iloc[:, 0])
mape = lambda df: mean_absolute_percentage_error(df.iloc[:, 1], df.iloc[:, 0])
r2 = lambda df: r2_score(df.iloc[:, 1], df.iloc[:, 0])
smape = lambda df: 200.0 * np.mean(np.abs(df.iloc[:, 1] - df.iloc[:, 0]) / (np.abs(df.iloc[:, 1]) + np.abs(df.iloc[:, 0])))
max_peak_error = lambda df: np.max(np.abs(df.iloc[:, 1] - df.iloc[:, 0]))
mean_n_peak_error = lambda df: np.mean(np.sort(np.abs(df.iloc[:, 1] - df.iloc[:, 0]))[-5:])
net_load_error = None

In [65]:
horizon = 48
season = "Winter"
model = "LightGBMModel"

eval_dict = dict_result_n_ahead

gt = eval_dict[horizon][season][2].pd_dataframe()
gt.columns = [f"gt_{col}" for col in gt.columns]


def get_eval_df(eval_dict, horizon, season, model):
    gt = eval_dict[horizon][season][2].pd_dataframe()
    df_preds = pd.concat([pred_batch.pd_dataframe() for pred_batch in eval_dict[horizon][season][0][model]], axis = 0)
    df_ = df_preds.join(gt, how = "left")
    return df_


,building_1_load_W,gt_building_1_load_W
datetime,,
2017-08-04 01:00:00,626.727248,717.5
2017-08-04 02:00:00,592.995474,725.0
2017-08-04 02:00:00,682.056207,725.0
2017-08-04 03:00:00,611.345215,737.5
2017-08-04 03:00:00,639.258732,737.5
...,...,...
2017-08-25 21:00:00,517.472075,622.5
2017-08-25 21:00:00,542.979070,622.5
2017-08-25 22:00:00,504.471080,575.0


In [74]:

mean_n_peak_error(df_)

1570.5801152556248

In [84]:
df_op = pd.read_csv("results/mpc_results/1_county/Los_Angeles/LightGBMModel_operations.csv", index_col=0)

In [85]:

import plotly.express as px
px.line(df_op)

In [97]:
df_op2 = pd.read_csv("/Users/nikolaushouben/Desktop/WattCast/data/results/mpc_results/1_county/Los_Angeles/operational_results.csv", index_col=0)

In [98]:
px.line(df_op2)